In [1]:
import pandas as pd
out_path = '../output'
df = pd.read_csv(r'../output/prepared_data.csv')
df = df[df['intent'] != 'Pad']
df

,sentence_id,language,intent,written,spoken,type,token_id,token,tag
0,0,en,OpenArea,show,show,plain,0,show,O
1,0,en,OpenArea,photo,photo,mycloudarea,1,photo,O
2,0,en,OpenArea,on,on,plain,2,on,O
3,0,en,OpenArea,mycloud,mycloud,plain,3,mycloud,O
4,1,en,OpenArea,show,show,plain,0,show,O
...,...,...,...,...,...,...,...,...,...
7609,1364,en,WatchNextEpisode,another,another,plain,0,another,O
7610,1364,en,WatchNextEpisode,episode,episode,plain,1,episode,O
7611,1365,en,WatchNextEpisode,one,one,plain,0,one,O
7612,1365,en,WatchNextEpisode,more,more,plain,1,more,O


In [2]:
dataset = df.groupby(['sentence_id'], as_index = False).agg({'token': list, 'tag':list})
texts, tags = dataset['token'].to_list(), dataset['tag'].to_list()

In [3]:
print(texts[1000], tags[1000], sep='\n')

['stop', 'device', 'family', 'room', 'in', 'the', 'kitchen']
['O', 'O', 'O', 'O', 'O', 'O', 'O']


In [4]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_tags, val_tags = train_test_split(texts, tags, test_size=.2)

In [5]:
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [7]:
from transformers import DistilBertTokenizerFast, AutoTokenizer, AutoModelWithLMHead
from transformers import DistilBertForTokenClassification
model = DistilBertForTokenClassification.from_pretrained('distilbert-base-uncased', num_labels=len(unique_tags))
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
tokenizer.save_pretrained('./pretrained/distilbert-base-uncased/')
model.save_pretrained('./pretrained/distilbert-base-uncased/')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN t

OSError: Can't load config for './local_model_directory/'. Make sure that:

- './local_model_directory/' is a correct model identifier listed on 'https://huggingface.co/models'

- or './local_model_directory/' is the correct path to a directory containing a config.json file



In [8]:
tokenizer = AutoTokenizer.from_pretrained('./pretrained/distilbert-base-uncased/')
model = AutoModelWithLMHead.from_pretrained('./pretrained/distilbert-base-uncased/')

/Users/taasuzh1/opt/miniconda3/envs/postprocessor/lib/python3.8/site-packages/transformers/models/auto/modeling_auto.py:966: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of the model checkpoint at ./pretrained/distilbert-base-uncased/ were not used when initializing DistilBertForMaskedLM: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing DistilBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceCla

In [14]:
from datasets import load_metric
metric = load_metric("seqeval")

In [21]:
type(metric)

datasets_modules.metrics.seqeval.81eda1ff004361d4fa48754a446ec69bb7aa9cf4d14c7215f407d1475941c5ff.seqeval.Seqeval

In [22]:
import pickle

# obj0, obj1, obj2 are created here...

# Saving the objects:
with open('metric_seqeval.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump(metric, f)

# Getting back the objects:
with open('metric_seqeval.pkl') as f:  # Python 3: open(..., 'rb')
    metric_ = pickle.load(f)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

In [9]:
train_encodings = tokenizer(train_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

In [10]:
import numpy as np

def encode_tags(tags, encodings):
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

train_labels = encode_tags(train_tags, train_encodings)
val_labels = encode_tags(val_tags, val_encodings)

In [11]:
import torch

class WNUTDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_encodings.pop("offset_mapping") # we don't want to pass this to the model
val_encodings.pop("offset_mapping")
train_dataset = WNUTDataset(train_encodings, train_labels)
val_dataset = WNUTDataset(val_encodings, val_labels)


In [12]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()


Step,Training Loss


KeyboardInterrupt: 